# Play Store App Reviews

A proposta é analisar um conjunto específico de avaliações de um aplicativo na Play Store e apresentar os resultados por meio de gráficos e tabelas. Essas representações visuais fornecerão informações atualizadas sobre os acontecimentos recentes relacionados ao aplicativo.

Esse script é ideal se você:

1. Quer saber o que os usuários estão falando do aplicativo da sua empresa/equipe.
2. Quer saber o que os usuários estão falando do aplicativo da concorrência.
3. Quer saber apenas por diversão.

#### Instalando e importando bibliotecas.

In [ ]:
%pip install google-play-scraper

In [ ]:
from google_play_scraper import app, reviews, Sort
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from wordcloud import WordCloud

#### Setando as variávies.

> Aqui você pode alterar as informações antes de rodar.

In [ ]:
APP_ID = "com.netflix.mediaclient" # O App ID que queremos analisar. (Você encontra isso na URL do App no Google Play Store)
LANGUAGE = "pt-br" # Língua.
COUNTRY = "br" # País.

TOTAL_ROWS_OF_DATA = 10000 # Aqui você coloca quantas avaliações você quer pegar, lembre-se que é a partir da avaliação mais recente.

#### Pegando os dados e organizando.

In [ ]:
app_infos = app(
    APP_ID,
    lang=LANGUAGE,
    country=COUNTRY
)

In [ ]:
results, _ = reviews(
    APP_ID,
    lang=LANGUAGE,
    country=COUNTRY,
    sort=Sort.NEWEST,
    count=TOTAL_ROWS_OF_DATA,
    filter_score_with=None,
)

In [ ]:
df = pd.DataFrame(np.array(results), columns=['review'])
df = df.join(pd.DataFrame(df.pop('review').tolist()))
df = df.rename(columns={
    "reviewId": "Review ID",
    "content": "Content",
    "score": "Score",
    "thumbsUpCount": "Thumbs Up Count",
    "reviewCreatedVersion": "Review Created Version",
    "at": "At",
    "replyContent": "Reply Content",
    "repliedAt": "Replied At",
    "appVersion": "App Version"
})
df = df.drop(["userName", "userImage"], axis=1)

### Informações gerais.

> Aqui, vamos fornecer uma visualização abrangente do comportamento dos nossos dados, o que facilitará a análise. É importante ter em mente o intervalo de datas ao realizar qualquer análise.

In [ ]:
average_thumbs_up = df['Thumbs Up Count'].mean()
response_percentage = (df['Reply Content'].notnull().sum() / len(df)) * 100
total_reviews = len(df)

df['At'] = pd.to_datetime(df['At'])
min_date = df['At'].min().strftime('%d/%m/%Y')
max_date = df['At'].max().strftime('%d/%m/%Y')
average_score = df['Score'].mean()

summary_df = pd.DataFrame({
    'Aplicativo em análise': [app_infos['title']],
    'Versão atual na Play Store': [app_infos['version']],
    'Pontuação na Play Store': ['{:.2f}'.format(app_infos['score'])],
    'Instalações na Play Store': [f"{app_infos['realInstalls']:,}"],
    'Total de últimas reviews que vamos analisar': [total_reviews],
    'Dos dados que pegamos, menor data de review': [min_date],
    'Dos dados que pegamos, maior data de review': [max_date],
    'Dos dados que pegamos, média das avaliações': ['{:.2f}'.format(average_score)],
    'Dos dados que pegamos, número médio de curtidas por review': ['{:.2f}'.format(average_thumbs_up)],
    'Dos dados que pegamos, porcentagem de reviews com resposta': [response_percentage]
})

summary_df = summary_df.T
summary_df.columns = ['Valor']
summary_df

### Pontuações.

> Através do gráfico abaixo, é possível visualizar as avaliações.

In [ ]:
score_counts = df['Score'].value_counts().reset_index()
score_counts.columns = ['Score', 'Count']

px.bar(score_counts, x='Score', y='Count', title='Número de avaliações por estrela.')

> No gráfico abaixo você irá saber qual o **saldo** das avaliações por dia do aplicativo. Positivo, negativo e neutro.

In [ ]:
df['At'] = pd.to_datetime(df['At'])
df['Data'] = df['At'].dt.date
df['Sentimento'] = pd.cut(df['Score'], bins=[0, 2, 3, 5], labels=['Negativo', 'Neutro', 'Positivo'])

grouped_df = df.groupby(['Data', 'Sentimento']).size().reset_index(name='Contagem')
fig = px.bar(grouped_df, x='Data', y='Contagem', color='Sentimento', title='Como anda o saldo das nossas avaliações por dia?',
            barmode='stack')

fig.add_annotation(
    text='São avaliações positivas maiores que 3 e negativas menores que 3. Neutras são 3.',
    xref='paper',
    yref='paper',
    x=0.5,
    y=1.1,
    showarrow=False,
    font=dict(size=12)
)

fig.show()

> No gráfico abaixo vamos conseguir ver as **versões** do aplicativo e suas respectivas avaliações.

In [ ]:
app_version_counts = df['App Version'].value_counts(normalize=True) * 100

filtered_app_versions = app_version_counts[app_version_counts >= 5]
filtered_df = df[df['App Version'].isin(filtered_app_versions.index)]

score_counts = filtered_df.groupby(['Score', 'App Version']).size().reset_index(name='Count')
score_counts = score_counts.sort_values('Score')

fig = px.bar(score_counts, x='Score', y='Count', color='App Version', title='Distribuição dos pontos por versão de aplicativo.',
            barmode='group')
fig.update_layout(xaxis_title='Avaliação', yaxis_title='Contagem')

fig.add_annotation(
    text='De todas nossas versões, estamos analisando apenas as que possuem mais dados. Ou seja, apenas versões mais avaliadas.',
    xref='paper',
    yref='paper',
    x=0.5,
    y=1.1,
    showarrow=False,
    font=dict(size=12)
)

fig.show()

### Comentários.

> Aqui vamos saber quais são os nossos **comentários mais curtidos**, as **reclamações** e os **elogios** dos usuários.

In [ ]:
QUANTIDADE_COMENTARIOS = 10 # Você pode editar isso! ;)

pd.set_option('display.max_colwidth', None)
top_reviews = df.sort_values(by='Thumbs Up Count', ascending=False).head(QUANTIDADE_COMENTARIOS)
top_reviews = top_reviews.reset_index(drop=True).drop(columns=['Review ID', 'Reply Content', 'Replied At'])
top_reviews = top_reviews.rename(columns={
    "Content": "Comentário",
    "Score": "Nota",
    "Thumbs Up Count": "Curtidas",
    "Review Created Version": "Versão do usuário",
    "At": "Avaliado em",
    "App Version": "Versão do app"
})
desired_order = ['Sentimento', 'Nota', 'Curtidas', 'Comentário', 'Versão do usuário', 'Versão do app', 'Data']
top_reviews = top_reviews.reindex(columns=desired_order)
top_reviews

> Núvem de palavras com os comentários dos usuários.

In [ ]:
text = ' '.join(df['Content'].astype(str))

wordcloud = WordCloud(width=800, height=400, background_color='white',
                    max_words=100, max_font_size=100, colormap='winter')
wordcloud.generate_from_text(text)

fig = go.Figure(data=go.Image(z=wordcloud.to_image()))
fig.update_layout(showlegend=False, xaxis_visible=False, yaxis_visible=False)

fig.show()